In [1]:
import requests
import pandas as pd
from datetime import datetime, date
import time
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')
import os

In [2]:
client_id = 'kIeG3SQnyatTNzP2SKF_eg'
secret_key = 'hnXPG4wASTbzxDwY9iW01HvG8_WrIg'

auth = requests.auth.HTTPBasicAuth(client_id, secret_key)

data = {
    'grant_type': 'password',
    'username': 'veochae',
    'password': 'Hold5489!!'
}

headers = {'User-Agent': 'MyAPI/0.0.1'}

res = requests.post('https://www.reddit.com/api/v1/access_token', 
                    auth = auth, 
                    data = data,
                    headers = headers)

token = res.json()['access_token']

headers['Authorization'] = f'bearer {token}'

In [7]:
def reddit_data(time_wanted):
    stopper = 0

    #initial set collection
    res = requests.get('https://oauth.reddit.com/r/Dreams/new',
                    headers = headers, params={'limit': '100'})
    
    df = pd.DataFrame()

    for post in res.json()['data']['children']:
        df = pd.concat([df,pd.DataFrame({'subreddit': post['data']['subreddit'],
                                                    'title': post['data']['title'],
                                                    'text': post['data']['selftext'],
                                                    'date': post['data']['created']},index=[0])], ignore_index= True)
    
    #further back collection
    latest_key = post['kind'] + '_' + post['data']['id']

    while df.tail(1)['date'][df.tail(1)['date'].index[0]] > datetime.timestamp(time_wanted):
        for req in range(100):
        
            res = requests.get('https://oauth.reddit.com/r/Dreams/new',
                                headers = headers, 
                                params={'limit': '100', 'after': latest_key})
            
            for post in res.json()['data']['children']:
                df = pd.concat([df,pd.DataFrame({'subreddit': post['data']['subreddit'],
                                                    'title': post['data']['title'],
                                                    'text': post['data']['selftext'],
                                                    'date': post['data']['created']},index=[0])], ignore_index= True)

            latest_key = post['kind'] + '_' + post['data']['id']
                
            print(f'{len(df)} rows collected')

            if len(df) >= 988:
                latest = df.tail(1)['date'][df.tail(1)['date'].index[0]]
                print("Data Collection Target Reached")
                print(f'{len(df)} rows collected')
                print(f'latest subreddit date: {datetime.fromtimestamp(latest)}')
                return df

            # time.sleep(5)
    else: 
        print("Date Limit Reached")
        print(f'{len(df)} rows collected')
        return df
    

In [8]:
df = reddit_data(datetime(2023, 1, 20, 00, 00, 00, 342380))

200 rows collected
300 rows collected
400 rows collected
500 rows collected
600 rows collected
700 rows collected
800 rows collected
900 rows collected
997 rows collected
Data Collection Target Reached
997 rows collected
latest subreddit date: 2024-02-29 14:02:18


In [14]:
from utils import task
import multiprocessing

def multiprocessing_function(text_data):
    
    print("**Data Filtering in Progress**: This Process would take about 2-3 Minutes!")
    try:
        with multiprocessing.Pool(processes=6) as pool:
            res = pool.starmap(task, enumerate(text_data)) 
    except Exception as e:
        print("exception in worker process", e)
        return text_data

    res.sort(key=lambda x: x[0])
    final_results = [result[1] for result in res]
    return final_results

In [15]:
df.text = multiprocessing_function(df.text)

**Data Filtering in Progress**: This Process would take about 2-3 Minutes!


In [18]:
df.text[1]

"I had a nightmare that I was in some sort of transparent pod and I had no skin on me at all, and no hands or feet, just muscle and bone truncated at the ankle and wrist, and I was still alive.  I guess that's how I feel as someone with a debilitating physical condition, kinda like a meat sack. Yeah, not one of my finer moments as a dreamer, for sure. "

In [12]:
print(Path.cwd())
os.chdir('../app')

/Users/veochae/Desktop/Dreams/data collection


In [ ]:
df= df.dropna()
df.to_csv("./raw_data.csv")